In [1]:
downsample = False

In [2]:
import os
import sys
import time
import argparse
import numpy as np
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
import pandas as pd
from torch.utils.data import DataLoader
from models.efficientdet import EfficientDet
from models.losses import FocalLoss
from datasets import Spine_dataset, get_augumentation, detection_collate
from utils import EFFICIENTDET
from tqdm import tqdm_notebook as tqdm
import cv2
import matplotlib.pyplot as plt

In [3]:
resume = None#'weights/checkpoint_efficientdet-d3_33.pth'
network = 'efficientdet-d5'
num_epochs = 40
batch_size = 2
num_worker = 4
num_classes = 1
device = [0]
grad_accumulation_steps = 1
learning_rate = 1e-4
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
save_folder = '../weights/'
image_root = 'boostnet_labeldata/data/'
csv_root = 'boostnet_labeldata/labels/'

In [4]:
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

In [5]:
def prepare_device(device):
    n_gpu_use = len(device)
    n_gpu = torch.cuda.device_count()
    if n_gpu_use > 0 and n_gpu == 0:
        print("Warning: There\'s no GPU available on this machine, training will be performed on CPU.")
        n_gpu_use = 0
    if n_gpu_use > n_gpu:
        print("Warning: The number of GPU\'s configured to use is {}, but only {} are available on this machine.".format(
            n_gpu_use, n_gpu))
        n_gpu_use = n_gpu
    list_ids = device
    device = torch.device('cuda:{}'.format(
        device[0]) if n_gpu_use > 0 else 'cpu')

    return device, list_ids

In [6]:
def get_state_dict(model):
    if type(model) == torch.nn.DataParallel:
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()
    return state_dict

In [7]:
checkpoint = []
if(resume is not None):
    resume_path = str(resume)
    print("Loading checkpoint: {} ...".format(resume_path))
    checkpoint = torch.load(
        resume, map_location=lambda storage, loc: storage)
    num_classes = checkpoint['num_class']
    network = checkpoint['network']

In [8]:
corner_df_train = pd.read_csv(csv_root+'training/landmarks.csv',header = None)
filename_df_train = pd.read_csv(csv_root+'training/filenames.csv',header = None)
boxes_df_train = pd.read_csv(csv_root+'training/train.csv')
boxes_df_train.label = 0 # All boxes same class??

In [9]:
corner_df_test = pd.read_csv(csv_root+'test/landmarks.csv',header = None)
filename_df_test = pd.read_csv(csv_root+'test/filenames.csv',header = None)
boxes_df_test = pd.read_csv('test.csv')
boxes_df_test.label = 0

In [10]:
train_dataset = Spine_dataset.SPINEDetection(image_root,boxes_df_train,corner_df_train,filename_df_train,transform=get_augumentation(phase = 'train',downsample = downsample),downsample = downsample)

In [11]:
val_dataset = Spine_dataset.SPINEDetection(image_root,boxes_df_test,corner_df_test,filename_df_test,transform=get_augumentation(phase = 'val',downsample = downsample),image_set='test', downsample = downsample)

In [12]:
test_dataset = Spine_dataset.SPINEDetection_test(transform=get_augumentation(phase = 'test',downsample = downsample))

In [13]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              num_workers=num_worker,
                              shuffle=True,
                              collate_fn=detection_collate,
                              pin_memory=True)

In [14]:
val_dataloader = DataLoader(val_dataset,
                              batch_size=batch_size,
                              num_workers=num_worker,
                              shuffle=False,
                              collate_fn=detection_collate,
                              pin_memory=True)

In [15]:
test_dataloader = DataLoader(test_dataset,
                              batch_size=batch_size,
                              num_workers=num_worker,
                              shuffle=False,
                              collate_fn=None,
                              pin_memory=True)

In [16]:
for idx, (images, annotations, corners) in enumerate(train_dataloader):
    print(idx ,images.shape, annotations.shape, corners.shape)
    break

[array([230., 302., 225., 296.,  34.,  49.,  72.,  96.]), array([220., 294., 211., 275.,  86., 109., 143., 169.]), array([209., 273., 189., 253., 153., 181., 202., 239.]), array([183., 250., 169., 234., 218., 252., 269., 300.]), array([160., 230., 145., 217., 287., 316., 346., 364.]), array([145., 215., 133., 205., 363., 377., 425., 430.]), array([135., 207., 135., 216., 446., 447., 513., 507.]), array([137., 216., 139., 221., 534., 517., 594., 580.]), array([141., 225., 154., 237., 620., 593., 684., 659.]), array([159., 240., 171., 251., 700., 672., 766., 741.]), array([176., 256., 186., 268., 788., 758., 858., 840.]), array([183., 270., 190., 288., 884., 860., 966., 934.]), array([ 196.,  293.,  203.,  316.,  995.,  958., 1058., 1027.]), array([ 212.,  317.,  220.,  326., 1091., 1057., 1161., 1128.]), array([ 222.,  328.,  227.,  337., 1188., 1153., 1265., 1242.]), array([ 234.,  358.,  235.,  360., 1290., 1268., 1372., 1349.]), array([ 235.,  363.,  244.,  364., 1393., 1385., 1465.,

[array([239., 140., 234., 146.,  40.,  42.,  73.,  82.]), array([231., 148., 236., 160.,  84., 108., 125., 156.]), array([239., 166., 255., 183., 139., 169., 183., 215.]), array([263., 191., 286., 211., 195., 229., 250., 280.]), array([293., 213., 319., 221., 258., 286., 322., 342.]), array([320., 219., 329., 217., 337., 351., 413., 410.]), array([328., 216., 317., 217., 440., 419., 509., 490.]), array([313., 215., 302., 204., 529., 498., 596., 569.]), array([288., 199., 271., 177., 615., 579., 687., 655.]), array([264., 175., 249., 145., 711., 672., 785., 741.]), array([241., 138., 222., 101., 806., 758., 885., 845.]), array([224., 107., 225.,  90., 869., 846., 946., 938.]), array([ 210.,   70.,  217.,   91.,  961.,  969., 1036., 1080.]), array([ 216.,   89.,  247.,  128., 1051., 1109., 1120., 1188.]), array([ 255.,  146.,  288.,  186., 1141., 1211., 1200., 1273.]), array([ 314.,  197.,  362.,  236., 1218., 1294., 1286., 1369.]), array([ 381.,  246.,  408.,  260., 1336., 1387., 1424.,

In [17]:
model = EfficientDet(num_classes=num_classes,
                     network=network,
                     W_bifpn=EFFICIENTDET[network]['W_bifpn'],
                     D_bifpn=EFFICIENTDET[network]['D_bifpn'],
                     D_class=EFFICIENTDET[network]['D_class'],
                     )

Loaded pretrained weights for efficientnet-b5


In [18]:
if(resume is not None):
    model.load_state_dict(checkpoint['state_dict'])
device, device_ids = prepare_device(device)
model = model.to(device)
if(len(device_ids) > 1):
    model = torch.nn.DataParallel(model, device_ids=device_ids)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
if(resume is not None):
    optimizer.load_state_dict(checkpoint['optimizer'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=3, verbose=True)
criterion = FocalLoss()

In [19]:
model.train()
best_loss = 100
df = pd.DataFrame(np.zeros((num_epochs,6)),columns = ["train_cls","train_bbox_loss","train_corner_loss","val_cls","val_bbox_loss","val_corner_loss"])
for epoch in range(num_epochs):
    print("{} epoch: \t start training....".format(epoch))
    start = time.time()
    result = {}
    total_loss = []
    bbox_losses = []
    cls_losses = []
    corner_losses = []
    optimizer.zero_grad()
    total_batches = len(train_dataloader)
    tk0 = tqdm(train_dataloader, total=total_batches)
    for idx, (images, annotations_bboxes, annotations_corners) in enumerate(tk0):
        images = images.to(device)
        annotations_bboxes = annotations_bboxes.to(device)
        annotations_corners = annotations_corners.to(device)
        classification, regression, corners, anchors = model(images)
        classification_loss, regression_loss, corner_loss= criterion(
            classification, regression, corners, anchors, annotations_bboxes, annotations_corners)
        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()
        corner_loss = corner_loss.mean()
        loss = classification_loss + regression_loss + corner_loss
        if bool(loss == 0):
            print('loss equal zero(0)')
            continue
        loss.backward()
        if (idx+1) % grad_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
            optimizer.step()
            optimizer.zero_grad()
        total_loss.append(loss.item())
        corner_losses.append(corner_loss.item())
        bbox_losses.append(regression_loss.item())
        cls_losses.append(classification_loss.item())
        tk0.set_postfix(loss=(np.mean(total_loss)))
    result = {
        'time': time.time() - start,
        'loss': np.mean(total_loss),
        'corner_loss': np.mean(corner_losses),
        'bbox_loss': np.mean(bbox_losses),
        'cls_loss': np.mean(cls_losses)
    }
    for key, value in result.items():
        print('    {:15s}: {}'.format(str(key), value))
    df.iloc[epoch,:3] = [np.mean(cls_losses),np.mean(bbox_losses),np.mean(corner_losses)] 
    torch.cuda.empty_cache()
    with torch.no_grad():
        start = time.time()
        result = {}
        total_loss = []
        bbox_losses = []
        cls_losses = []
        corner_losses = []
        optimizer.zero_grad()
        total_batches = len(test_dataloader)
        tk0 = tqdm(val_dataloader, total=total_batches)
        for idx, (images, annotations_bboxes, annotations_corners) in enumerate(tk0):
            images = images.to(device)
            annotations_bboxes = annotations_bboxes.to(device)
            annotations_corners = annotations_corners.to(device)
            classification, regression, corners, anchors = model(images)
            classification_loss, regression_loss, corner_loss= criterion(
                classification, regression, corners, anchors, annotations_bboxes, annotations_corners)
            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()
            corner_loss = corner_loss.mean()
            loss = classification_loss + regression_loss + corner_loss
            if bool(loss == 0):
                print('loss equal zero(0)')
                continue
            total_loss.append(loss.item())
            corner_losses.append(corner_loss.item())
            bbox_losses.append(regression_loss.item())
            cls_losses.append(classification_loss.item())
            tk0.set_postfix(loss=(np.mean(total_loss)))
        result = {
            'time': time.time() - start,
            'loss': np.mean(total_loss),
            'corner_loss': np.mean(corner_losses),
            'bbox_loss': np.mean(bbox_losses),
            'cls_loss': np.mean(cls_losses)
        }
        for key, value in result.items():
            print('    {:15s}: {}'.format(str(key), value))
            
    scheduler.step(np.mean(total_loss))
    df.iloc[epoch,3:] = [np.mean(cls_losses),np.mean(bbox_losses),np.mean(corner_losses)]
    df.to_csv('../logs/d5-resize-corner-(1536,512).csv')
    torch.cuda.empty_cache()
    arch = type(model).__name__
    state = {
        'arch': arch,
        'num_class': num_classes,
        'network': network,
        'state_dict': get_state_dict(model),
        'optimizer': optimizer.state_dict()
    }
    if(best_loss>np.mean(corner_losses)):
        best_loss = np.mean(corner_losses)
        torch.save(state, '../weights/d5-resize-corner-(1536,512).pth')

0 epoch: 	 start training....


/home/arnav0400/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


ValueError: Caught ValueError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/home/arnav0400/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/arnav0400/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/arnav0400/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/arnav0400/Spine-Project/EfficientDet.Pytorch/datasets/Spine_dataset.py", line 40, in __getitem__
    augmentation = self.transform(**annotation)
  File "/home/arnav0400/anaconda3/lib/python3.7/site-packages/albumentations/core/composition.py", line 174, in __call__
    p.preprocess(data)
  File "/home/arnav0400/anaconda3/lib/python3.7/site-packages/albument

In [ ]:
model.is_training = False

In [ ]:
def vis(dataloader,split = 'test'):
    if split=='test':
        for idx, (images) in enumerate(dataloader):
            images = images.to(device)
            image = images[0]
            classification_score, category, boxes, corners = model(images)
            break
    else:
        for idx, (images, annotations, corners) in enumerate(dataloader):
            image = images[0]
            images = images.to(device)
#             classification_score, category, boxes, corners = model(images)
            break
    corners = corners.detach().cpu().numpy()
    boxes = boxes.detach().cpu().numpy()
    image = image.detach().cpu().numpy()
    image = image.transpose(1,2,0)
    boxes = boxes.astype(np.int16)
    for box in boxes:
        start = (box[0],box[1])
        end = (box[2],box[3])
        image = cv2.rectangle(image, start, end, (255,0,0), 10)
    plt.scatter(corners[:,:4],corners[:,4:])
    plt.imshow(image.get())

In [ ]:
for idx, (images, annotations, corners) in enumerate(train_dataloader):
    image = images[0]
    image = image.detach().cpu().numpy()
    image = image.transpose(1,2,0)
    corners = corners[0].detach().cpu().numpy()
    corners = corners.astype(np.int16)
    plt.scatter(corners[:,:4],corners[:,4:])
    plt.imshow(image)
    break

In [ ]:
vis(train_dataloader,'a')

In [ ]:
vis(test_dataloader)

In [ ]:
def get_predictions(dataloader = val_dataloader):
    corners = []
    clipped_corners = []
    
    # Predictions from model
    for idx, (image, annotation, corner) in enumerate(val_dataloader):
        image = image.to(device)
        classification_score, category, pred_boxes, pred_corner = model(image)
        classification_score = classification_score.detach().cpu().numpy()
        pred_corner = pred_corner.detach().cpu().numpy()
        if classification_score.shape[0]>17:
            ind = np.argpartition(classification_score, -17)[-17:]
            corners.append(pred_corner[ind])
        else:
            corners.append(pred_corner)
    
    #Making all predictions 17
    for corner in corners:
        if corner.shape[0]==17:
            clipped_corners.append(corner)
        else:
            num_repeat = 17-corner.shape[0]
            repeat = corner[-1].reshape(1,8)
            for i in range(num_repeat):
                corner = np.append(corner,repeat,axis=0)
            clipped_corners.append(corner)
    
    clipped_corners = np.array(clipped_corners)
    
    #Reshaping and saving csv
    val_landmarks = np.zeros((len(clipped_corners),136))
    for i in range(len(clipped_corners)):
        val_landmarks[i,:68] = clipped_corners[i,:,:4].reshape(1,68)/768
        val_landmarks[i,68:] = clipped_corners[i,:,4:].reshape(1,68)/1408
    
    val_preds = pd.DataFrame(val_landmarks)
    val_preds.to_csv('val_preds.csv',header = None,index = False)
    return val_preds

In [ ]:
get_predictions()

In [ ]:
import numpy as np
import pandas as pd

## find the SMAPE error
def smape(y_true, y_pred):
    numerator = np.sum(np.abs(y_true-y_pred), axis=1)
    denominator = np.sum(np.abs(y_true+y_pred), axis=1)
    smape_val = np.mean(numerator/ denominator) * 100
    return smape_val



true_csv_path = "boostnet_labeldata/labels/test/angles.csv"
pred_csv_path = "boostnet_labeldata/labels/test/test_angles_polyfit.csv"
true_angles = pd.read_csv(true_csv_path, header=None).values
pred_angles = pd.read_csv(pred_csv_path, header=None).values
print (smape(true_angles, pred_angles))